# Toronto Neighborhood - Web Scraping from Wikipedia page

In [2]:
#import required libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup 

## 1. Scraping table data and converting to pandas DataFrame

In [3]:
#defining the URL
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
#Use BeautifulSoup library to get the table data from web-page and store as a Pandas DataFrame 
results = requests.get(url)
soup = BeautifulSoup(results.content, 'lxml')
table = soup.find_all('table')[0]
df_tor = pd.read_html(str(table))[0]
df_tor.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [5]:
#Check the df shape
df_tor.shape

(180, 3)

## 2. Drop rows that contains 'Not assigned' or NA value in 'Borough'

In [6]:
#Drop rows that does not contain any data at all
df_tor = df_tor.dropna(how='all')
df_tor.shape

(180, 3)

Does not contain any empty rows. Next check how many rows contain borough 'Not assigned'

In [7]:
df_tor['Borough'].value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Mississauga          1
Name: Borough, dtype: int64

77 rows contain Not assigned value in Borough. We need to exclude these 77 rows from our df.

In [8]:
#Remove rows Unassigned rows
df_tor.drop(df_tor[df_tor['Borough'] == 'Not assigned'].index, inplace=True)
df_tor.shape

(103, 3)

## 3. Find Null/NA/Not Assigned values in Neighborhood column and replace with respective Borough value.
Find how many rows do not have Neighborhood data.

In [9]:
#Check if any column has null values
df_tor.isnull().sum()

Postal code     0
Borough         0
Neighborhood    0
dtype: int64

In [10]:
#Check if any cell in Neighborhood contains the string 'Not' to find if any cell contains Not assigned value.
df_tor['Neighborhood'].str.contains(r'Not').value_counts()

False    103
Name: Neighborhood, dtype: int64

In [11]:
#Check if any cell in Neighborhood column contains NA values
df_tor['Neighborhood'].isna().value_counts()

False    103
Name: Neighborhood, dtype: int64

All cells in Neighborhood contains a valid value, hence it is not required to copy values from 'Borough' column.

## 4. Merge Neighborhood values for duplicate Postal code values.
Check if any values in Postal Code column is repeated.

In [12]:
#Checking if any values are repeated in Postal Code column, if repeated the value_counts method will return a value more than 1. 
#All rows returned True, indicating that no values are repeated in Postal Code column.

(df_tor['Postal code'].value_counts()==1).value_counts()

True    103
Name: Postal code, dtype: int64

In [13]:
#Cross-verifying using duplicated method. Duplicated method sets True value for duplicated rows. 
#The output does not contain any rows, indicating that there are no duplicate values.

df_tor[df_tor.duplicated(['Postal code'])].shape

(0, 3)

No duplicate values are present in Postal code column, hence no need to combine Neghborhood values. However on looking at the Wikipedia table found that the Postal Codes with multiple neighborhoods are already merged with a '/' in between. Example below.

In [14]:
df_tor[df_tor['Neighborhood'].str.contains(r'/')].shape

(55, 3)

55 rows contain ' / ' in Neighborhood column, indicating that merged data. Need to replace ' / ' with a comma(.) as stated in the instructions. <br> Now let's check any cells already contain comma(,).

In [15]:
df_tor[df_tor['Neighborhood'].str.contains(r',')].shape

(1, 3)

1 cell already has comma(,) in between different Neighborhoods.

In [16]:
#Replace '/' with a comma(,) for values in Neighorhood column
df_tor['Neighborhood']= df_tor['Neighborhood'].str.replace("/", ",", case = False)
df_tor.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [17]:
#Check how many columns now contain comma(,) as the value separator
df_tor[df_tor['Neighborhood'].str.contains(r',')].shape

(56, 3)

## Summary of DataFrame
Now the dataframe is ready as requried and fulfills the below condition:<br>
- Contains only 3 columns - 'Postal code', 'Borough', 'Neighborhood'
- Does not contain 'Not assigned' values in 'Borough' or 'Neighborhood'
- No cell has assigned 'Borough' but unassigned 'Neighborhood'.
- Duplicate Postal codes are merged into single row with multiple 'Neighborhood' values separated using comma(,)<br>

Finally will check the shape of the DataFrame.


In [35]:
#Check the Shape of df
df_tor.shape

(103, 3)

In [34]:
# The code was removed by Watson Studio for sharing.

{'file_name': 'Toronto_Neighborhood.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'courseracapstoneprojectprabhakara-donotdelete-pr-aka7o9ldbs0rav',
 'asset_id': '5c1ea04c-5f97-4df3-8346-a2470879fb3e'}